In [1]:
from const import Token_1
import telebot
from telebot import types
import sqlite3 as sq
import datetime
import re


bot = telebot.TeleBot(Token_1, parse_mode=None)

"""with open("BotT.txt", "r") as file:
    TOKEN = file.read().strip()

bot = telebot.TeleBot(TOKEN, parse_mode=None)"""

# create an empty dictionary to store user data
user_data = {}

# create date: 05.03.2023 and time: 12:00
date_1 = datetime.date(2023, 3, 5)
time_1 = datetime.time(12,0)

table_name = f"tour_at_{date_1.strftime('%d_%m_%Y')}_{time_1.strftime('%H_%M')}"#.replace('-', '_')
#SQL doesn`t allow '-', so i have to replace it

# open file with database
with sq.connect('mus_schedule.db', check_same_thread=False) as connect:
  cursor = connect.cursor()

#
if cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}';").fetchone():
  cursor.execute(f"SELECT SUM(quantity) FROM {table_name}")
  total_quantity = cursor.fetchone()[0] # return tuple with 'Sum'
  if total_quantity is not None and total_quantity > 50:
  # create new table with group for the next hour
    datetime_1 = datetime.datetime.combine(date_1, time_1) + datetime.timedelta(hours=1) #combine to convert in 'date' object 
    time_1 = datetime_1.time() #cause adding hours is forbidden
  

table_name = f"tour_at_{date_1.strftime('%d_%m_%Y')}_{time_1.strftime('%H_%M')}"

cursor.execute(f"""
            CREATE TABLE IF NOT EXISTS {table_name} (
            name STRING,
            quantity INTEGER,
            contact STRING UNIQUE
            );   """)
connect.commit()

@bot.message_handler(commands=['omet','start']) # comands that responded
#@bot.message_handler(content_types=['text','sticker'])
def respond(message):
  
# photo_mus = types.InputMediaPhoto('https://scontent-ams2-1.xx.fbcdn.net/v/t1.6435-9/193684250_1184653448716244_1549048136683557681_n.jpg?_nc_cat=108&ccb=1-7&_nc_sid=8bfeb9&_nc_ohc=7yIjZg512oIAX-92GYU&_nc_ht=scontent-ams2-1.xx&oh=00_AfAudupE6CtYIyfwv4wymPOXc7HrylZBNvlL-KIs7B_Q3Q&oe=643A8642')
    
  text_menu=f"<b>{message.from_user.first_name}</b>.,виберіть та натисніть одну з кнопок нижче\n\
           1.Зареєструватись на найближчу екскурсію.\n\
           2.Відвідати наш сайт."
           #3.Залишити відгук відносно роботи електротранспорту."
  markup=types.InlineKeyboardMarkup(row_width=1) #, resize_keyboard=True)
    
      
  button_1=types.InlineKeyboardButton(f"1.Екскурсія\
    {date_1.strftime ('%d-%m-%Y')} о {time_1}",callback_data='schedule')
     #photo = photo.read(),\
      #photo = photo_mus)
  with open('photo_2023-03-13_10-20-36.jpg', 'rb') as photo:
    bot.send_photo(message.chat.id, photo, reply_markup=markup)
    
  button_2=types.InlineKeyboardButton("2.Відвідати офіційну сторінку КП 'OMET'",\
    url="https://oget.od.ua",callback_data='site')
  #button_3=types.InlineKeyboardButton("3.Залишити відгук.", callback_data='respond')
  markup.add(button_1,button_2)#,button_3)

  bot.send_message(message.chat.id, text_menu, reply_markup=markup, parse_mode='html') #,resize_keybord=True)


def respond_buttons(message):
    markup = types.InlineKeyboardMarkup(row_width=3)

    text_button = types.InlineKeyboardButton("Текстовий відгук", callback_data='text_respond')
    audio_button = types.InlineKeyboardButton("Аудіо відгук", callback_data='audio_respond')
    video_button = types.InlineKeyboardButton("Відео відгук", callback_data='video_respond')

    markup.add(text_button, audio_button, video_button)
    
    bot.send_message(message.chat.id, "Оберіть форму вашого звернення:", reply_markup=markup)

@bot.callback_query_handler(func=lambda call:True)
def callback(call):
  if call.message: # check if button pushed
    if call.data == 'schedule':
      bot.register_next_step_handler(call.message, save_quantity)
      bot.send_message(call.message.chat.id, "Напишіть цифрами (від 1 до 50) кількість людей,\
              які планують відвідати музей КП 'ОМЕТ'\n\
              (наприклад:'12' - бот ідентифікує лише цифрову складову у цьому повідомленні)")
      
    if call.data == 'respond':
      respond_buttons(call.message)
    if call.data == 'text_respond':
          bot.register_next_step_handler(call.message, save_text_respond)
          bot.send_message(call.message.chat.id, "Введіть текстовий відгук\n\
            (нагадуємо, що в повідомленні телеграма може вміститись близько 4000 символів)")
    if call.data == 'audio_respond':
          bot.register_next_step_handler(call.message, save_audio_respond)
          bot.send_message(call.message.chat.id, "Надішліть аудіо відгук\n\
            тривалістю до 1 хвилини")
    if call.data == 'video_respond':
          bot.register_next_step_handler(call.message, save_video_respond)
          bot.send_message(call.message.chat.id, "Надішліть відео відгук\n\
            тривалістю до 1 хвилини")

def save_text_respond(message):
    # Збереження текстового відгука
    text_from_user = message

"""Номер інформаційного центру 717 54 54 або 788 688 6"""

# How to make a function that will send telegram message via mail?


def save_audio_respond(message):
    # Збереження аудіо відгука
    pass

def save_video_respond(message):
    # Збереження відео відгука
    pass


def give_respond(message):
  pass

def save_quantity(message):
  quantity = message.text.strip()  # remove extra spaces from text 
  
  user_data['name'] = message.from_user.first_name
    
  # check if quantity is real number
  if not quantity.isdigit():
    bot.send_message(message.chat.id, "Невірно введена кількість")
    bot.register_next_step_handler(message, save_quantity)  # waiting for next message
    return
  
  # check if quantity more than 50
  if int(quantity) > 50:
    bot.send_message(message.chat.id, "Зв'яжіться із директором музею за телефоном 050 399 42 11 для обговорення окремої\
      екскурсії для такої великої кількості людей.\n\
      Якщо хочете повернутись в головне меню напишіть команду '/start' або '/omet'.")
    return
  
  # separate the numerical part from the text, if there is one
  digits_str = ''.join(re.findall('\d+', quantity))
  digits_int = int(digits_str) if digits_str else None
  
  user_data['quantity'] = digits_int
  
  bot.send_message(message.chat.id, f"Кількість відвідувачів {digits_int} збережено")
  
  bot.send_message(message.chat.id, "Надішліть будь ласка ваш номер телефону для зв'язку з вами,\n\
    наприклад: 0661234567 або 7562598 , без пробілів та інших символів крім цифр")
  
  bot.register_next_step_handler(message, ask_phone)
                   


def ask_phone(message):
  global table_name
  phone_number = message.text.strip()  # remove extra spaces from text
  int_phone = int(''.join(re.findall('\d+', phone_number)))
  
  # check if phone number is valid
  #if not phone_number.isdigit() or not 6 <= len(phone_number) <= 13:
  if not re.match(r'^\+?\d{6,13}$', phone_number):
    # '^' - begin of str; '\+?' - '+' could be included or not; '\d' - indicate any number, '{6,13} - quantity'; '$' - end of a str.
    bot.send_message(message.chat.id, "В номері невідповідна кількість цифр або він містить недопустимі символи")
    bot.register_next_step_handler(message, ask_phone)  # waiting for next message
    return
  
  with sq.connect('mus_schedule.db', check_same_thread=False) as connect:
    cursor = connect.cursor()
  
  # check if phone number and name is unique
    cursor.execute(f"SELECT COUNT(*) FROM {table_name} WHERE contact =? Or name =?", (int_phone, user_data['name']))
    count = cursor.fetchone()[0]
    if count > 0:
      bot.send_message(message.chat.id, "Номер телефону або користувача вже зареєстрований на цю екскурсію")
      bot.register_next_step_handler(message, ask_phone)  # waiting for next message
      return
  
  user_data['phone'] = int_phone
  
  cursor.execute(f"Insert Or Replace Into {table_name} (name, quantity, contact) VALUES (?, ?, ?)",\
    (user_data['name'], user_data['quantity'], user_data['phone']))
  connect.commit()
  
  connect.close()

  bot.send_message(message.chat.id, f"Ваш контакт збережено. Будемо раді бачити вас у музеї, гарного дня!\
    Якщо хочете повернутись в головне меню натисніть-напишіть команду '/start' або '/omet'")
  
  #bot.register_next_step_handler(message, respond)

  bot.register_next_step_handler(message, respond)

bot.infinity_polling()

2023-03-24 20:13:28,245 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2023-03-24 20:13:28,246 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"
